In [1]:
#importing and exporting data modules
from simple_salesforce import Salesforce
import gspread
import requests
from io import StringIO


import os
import logging
import gspread
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload



#data manipulation 
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


#plotting
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px


In [2]:
sf = Salesforce(username='bchen@envoy.com',password='TasksandEvents1', security_token='nQWlT8vNdnJwxwtfpS1ic4Z7O')
gc = gspread.service_account(filename='/Users/bchen/Downloads/gspread-428120-06947c66447d.json')
sht1 = gc.open_by_key('1A6Q8dvoWwLi26tnQQoEo6ZnekvZBQbpgCyP20iQV6Ug')


In [3]:
sf_instance = 'https://envoy.my.salesforce.com/' #Your Salesforce Instance URL
reportId = '00OUO000009IZVD2A4' # add report id
export = '?isdtp=p1&export=1&enc=UTF-8&xf=csv'
sfUrl = sf_instance + reportId + export
response = requests.get(sfUrl, headers=sf.headers, cookies={'sid': sf.session_id})
download_report = response.content.decode('utf-8')
df = pd.read_csv(StringIO(download_report))

/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_86043/3072986056.py:7: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(download_report))


In [4]:
# Your existing functions:
def get_quarter_info(current_date):
    fiscal_year = current_date.year if current_date.month >= 2 else current_date.year - 1

    if datetime(current_date.year, 2, 1) <= current_date < datetime(current_date.year, 5, 1):
        quarter = f"{fiscal_year + 1}-Q1"
        quarter_start_date = datetime(current_date.year, 2, 1)
        quarter_end_date = datetime(current_date.year, 4, 30)
    elif datetime(current_date.year, 5, 1) <= current_date < datetime(current_date.year, 8, 1):
        quarter = f"{fiscal_year + 1}-Q2"
        quarter_start_date = datetime(current_date.year, 5, 1)
        quarter_end_date = datetime(current_date.year, 7, 31)
    elif datetime(current_date.year, 8, 1) <= current_date < datetime(current_date.year, 11, 1):
        quarter = f"{fiscal_year + 1}-Q3"
        quarter_start_date = datetime(current_date.year, 8, 1)
        quarter_end_date = datetime(current_date.year, 10, 31)
    else:
        # Q4: Ends on Jan 31 of the following year
        if current_date.month == 1:
            quarter = f"{fiscal_year + 1}-Q4"
            quarter_start_date = datetime(current_date.year - 1, 11, 1)
            quarter_end_date = datetime(current_date.year, 1, 31)
        else:
            quarter = f"{fiscal_year + 1}-Q4"
            quarter_start_date = datetime(current_date.year, 11, 1)
            quarter_end_date = datetime(current_date.year + 1, 1, 31)
    return quarter, quarter_start_date, quarter_end_date

def compute_week_boundaries_for_quarter(quarter_start_date, quarter_end_date):
    total_days = (quarter_end_date - quarter_start_date).days + 1
    start_wd = quarter_start_date.weekday()  # Monday=0,...,Sunday=6

    # Fixed 11 weeks in the middle: 11*7 = 77 days
    leftover = total_days - 77
    solutions = []
    half = leftover / 2.0

    for W1 in range(2, 13):
        W13 = leftover - W1
        if 4 <= W13 <= 12 and ((start_wd + W1) % 7 == 4):
            solutions.append((W1, W13))

    if not solutions:
        raise ValueError(f"Could not find a valid week distribution for quarter starting {quarter_start_date}.")

    solutions.sort(key=lambda x: abs(x[0] - half))
    best_W1, best_W13 = solutions[0]

    week_lengths = [best_W1] + [7]*11 + [best_W13]
    week_boundaries = []
    start_of_week = quarter_start_date
    for length in week_lengths:
        end_of_week = start_of_week + timedelta(days=length - 1)
        week_boundaries.append((start_of_week, end_of_week))
        start_of_week = end_of_week + timedelta(days=1)
    return week_boundaries

# Helper function to break down a single date
def breakdown_date(date_val):
    if pd.isnull(date_val):
        return pd.Series({
            'Quarter': None,
            'Week_of_Quarter': None,
            'Month': None,
            'Day_of_Week': None,
            'Day_Name': None
        })
    # Use your custom logic for quarter and week
    quarter, q_start, q_end = get_quarter_info(date_val)
    week_boundaries = compute_week_boundaries_for_quarter(q_start, q_end)
    week_of_quarter = None
    for i, (wstart, wend) in enumerate(week_boundaries):
        if wstart <= date_val <= wend:
            week_of_quarter = i + 1
            break
    return pd.Series({
        'Quarter': quarter,
        'Week_of_Quarter': week_of_quarter,
        'Month': date_val.month,
        'Day_of_Week': date_val.weekday(),       # Monday=0, Sunday=6
        'Day_Name': date_val.strftime("%A")
    })


def get_last_completed_quarters(n, today=None):
    """
    Returns a list of the last n completed quarters (as strings) based on the custom fiscal calendar.
    For example, if today is 03/06/2025 (active quarter = "2026-Q1"), the most recent completed quarter is "2025-Q4".
    """
    if today is None:
        today = datetime.today()
    current_quarter, q_start, _ = get_quarter_info(today)
    last_date = q_start - timedelta(days=1)
    quarters = []
    for _ in range(n):
        q, q_start, _ = get_quarter_info(last_date)
        quarters.append(q)
        last_date = q_start - timedelta(days=1)
    return quarters[::-1]

def compute_day_of_quarter(date_val):
    if pd.isnull(date_val):
        return pd.Series({'Day_of_Quarter': np.nan, 
                          'Total_Days_in_Quarter': np.nan, 
                          'Pct_Day': np.nan})
    quarter, q_start, q_end = get_quarter_info(date_val)
    day_of_quarter = (date_val - q_start).days + 1
    total_days = (q_end - q_start).days + 1
    pct_day = day_of_quarter / total_days * 100
    return pd.Series({'Day_of_Quarter': day_of_quarter, 
                      'Total_Days_in_Quarter': total_days, 
                      'Pct_Day': pct_day})

def compute_cumulative_raw(df_in):
    grouped = df_in.groupby(['Quarter', 'Pct_Day_Bin'])['ARR Change'].sum().reset_index()
    pivot = grouped.pivot(index='Pct_Day_Bin', columns='Quarter', values='ARR Change').fillna(0)
    full_index = pd.Index(range(0, 101), name='Pct_Day_Bin')
    pivot = pivot.reindex(full_index, fill_value=0)
    cum = pivot.cumsum()
    return cum


def compute_cumulative_raw_pipegen(df_in):
    # For Pipegen: group by Pipegen_Quarter and Pipegen_Pct_Day_Bin using ARR Change
    grouped = df_in.groupby(['Pipegen_Quarter', 'Pipegen_Pct_Day_Bin'])['ARR Change'].sum().reset_index()
    pivot = grouped.pivot(index='Pipegen_Pct_Day_Bin', columns='Pipegen_Quarter', values='ARR Change').fillna(0)
    full_index = pd.Index(range(0, 101), name='Pipegen_Pct_Day_Bin')
    pivot = pivot.reindex(full_index, fill_value=0)
    cum = pivot.cumsum()
    return cum


In [5]:
# Assume you have loaded your SFDC report into a DataFrame called df
# For example: df = pd.read_csv("sfdc_report.csv")

# List of date columns to enhance
date_columns = ['Created Date', 'SQO Date', 'SAO Date', 'Timestamp: Solution Validation', 'Close Date']

# Convert each to datetime
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Process each date column and merge the breakdown back into df
for col in date_columns:
    # Apply the breakdown function to the column
    breakdown_df = df[col].apply(breakdown_date)
    # Rename the resulting columns to include the original column name as prefix
    breakdown_df = breakdown_df.rename(columns={
        'Quarter': f'{col}_Quarter',
        'Week_of_Quarter': f'{col}_Week_of_Quarter',
        'Month': f'{col}_Month',
        'Day_of_Week': f'{col}_Day_of_Week',
        'Day_Name': f'{col}_Day_Name'
    })
    # Concatenate the breakdown info back into the main DataFrame
    df = pd.concat([df, breakdown_df], axis=1)

# At this point, df contains your original columns plus the breakdowns for each date field.
# You can then save your master report:
df.to_csv("master_report.csv", index=False)


In [6]:
# 1) Get your worksheet
worksheet = sht1.worksheet("Data")

# 0) Clear the existing sheet so you start fresh
worksheet.clear()

# 2) Copy your DataFrame
data_to_update = df.copy()

# 3) Ensure headers are strings
data_to_update.columns = data_to_update.columns.astype(str)

# 4) Trim all whitespace from strings
data_to_update = data_to_update.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# 5) Turn any “only‐spaces” cell into an empty string
data_to_update = data_to_update.replace(r'^\s*$', '', regex=True)

# 6) Convert Timestamps to YYYY-MM-DD
data_to_update = data_to_update.applymap(
    lambda x: x.strftime('%Y-%m-%d') if isinstance(x, pd.Timestamp) else x
)

# 7) Replace NaN, ±inf with empty string
data_to_update = data_to_update.fillna('').replace([np.inf, -np.inf], '')

# 8) Build the list of lists, but map '' → None so Google Sheets sees a true blank
headers = data_to_update.columns.tolist()
rows = data_to_update.values.tolist()
values = [headers]
for row in rows:
    clean_row = [None if (isinstance(cell, str) and cell == '') else cell for cell in row]
    values.append(clean_row)

# 9) Update the sheet
worksheet.update(values, 'A1')

/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_86043/1076719585.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_to_update = data_to_update.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_86043/1076719585.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_to_update = data_to_update.applymap(


{'spreadsheetId': '1A6Q8dvoWwLi26tnQQoEo6ZnekvZBQbpgCyP20iQV6Ug',
 'updatedRange': 'Data!A1:BH35128',
 'updatedRows': 35128,
 'updatedColumns': 60,
 'updatedCells': 1838597}

In [7]:
#SQLs# --- 
#1) GET THE LAST 5 QUARTERS (4 completed + active) ---
today = datetime.today()
active_quarter = get_quarter_info(today)[0]
last_4 = get_last_completed_quarters(4, today)
quarters_5 = last_4 + [active_quarter]

# --- 2) SLIM DOWN YOUR DATAFRAME ---
df_pivot = df[[
    'Created Date_Quarter',
    'Source',
    'Segment - historical',
    'Bookings Type'
]].copy()

# Map any non-standard source into "Other"
allowed_src = ['AE','BDR','Channel','Marketing','Success']
df_pivot['Source'] = df_pivot['Source'].where(df_pivot['Source'].isin(allowed_src), 'Other')

# Filter to exactly the segments & booking types you want
allowed_segs  = ['Enterprise','Mid Market','SMB']
allowed_types = ['Expansion','New Business']
df_pivot = df_pivot[
    df_pivot['Segment - historical'].isin(allowed_segs) &
    df_pivot['Bookings Type'].isin(allowed_types)
]

# --- 3) BUILD A STATIC ROW INDEX ---
rows = []
for src in ['AE','BDR','Channel','Marketing']:
    for seg in allowed_segs:
        for bt in allowed_types:
            rows.append((src, seg, bt))
        rows.append((src, seg, f'{seg} Total'))
    rows.append((src, '', f'{src} Total'))

rows.append(('Other','','Other Total'))

for seg in allowed_segs:
    for bt in allowed_types:
        rows.append(('Success', seg, bt))
    rows.append(('Success', seg, f'{seg} Total'))
rows.append(('Success','','Success Total'))

rows.append(('','','Grand Total'))

index = pd.MultiIndex.from_tuples(
    rows,
    names=['Source','Segment - historical','Bookings Type']
)

# --- 4) INIT PIVOT SHELL ---
pivot = pd.DataFrame(
    0,
    index=index,
    columns=quarters_5 + ['Grand Total']
)

# --- 5) FILL THE DETAIL COUNTS ---
grp = df_pivot.groupby(
    ['Source','Segment - historical','Bookings Type','Created Date_Quarter']
).size()

for (src, seg, bt, q), cnt in grp.items():
    if (src,seg,bt) in pivot.index and q in quarters_5:
        pivot.at[(src,seg,bt), q] = cnt

# --- 6) SEGMENT SUBTOTALS ---
grp_seg = df_pivot.groupby(
    ['Source','Segment - historical','Created Date_Quarter']
).size()

for (src, seg, q), cnt in grp_seg.items():
    row = (src, seg, f'{seg} Total')
    if row in pivot.index and q in quarters_5:
        pivot.at[row, q] = cnt

# --- 7) SOURCE SUBTOTALS ---
grp_src = df_pivot.groupby(['Source','Created Date_Quarter']).size()
for (src, q), cnt in grp_src.items():
    row = (src, '', f'{src} Total')
    if row in pivot.index and q in quarters_5:
        pivot.at[row, q] = cnt

# --- 8) GRAND TOTAL PER QUARTER ---
grp_all = df_pivot.groupby('Created Date_Quarter').size()
for q, cnt in grp_all.items():
    if q in quarters_5:
        pivot.at[('', '', 'Grand Total'), q] = cnt

# --- 9) FINAL GRAND TOTAL COLUMN ---
pivot['Grand Total'] = pivot[quarters_5].sum(axis=1)

# --- 10) RESET INDEX FOR EXPORT / DISPLAY ---
pivot_display = pivot.reset_index()



In [8]:

# 1) Get your worksheet
worksheet = sht1.worksheet("SQLs")

# 2) Prepare the DataFrame you want to push
data_to_update = pivot_display.copy()  # pivot_display from the previous step

# 3) Ensure headers are strings
data_to_update.columns = data_to_update.columns.astype(str)

# 4) Trim all whitespace from strings
data_to_update = data_to_update.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# 5) Turn any “only‐spaces” cell into an empty string
data_to_update = data_to_update.replace(r'^\s*$', '', regex=True)

# 6) Convert any Timestamps to YYYY-MM-DD
data_to_update = data_to_update.applymap(
    lambda x: x.strftime('%Y-%m-%d') if hasattr(x, 'strftime') else x
)

# 7) Replace NaN, ±inf with empty string
data_to_update = data_to_update.fillna('').replace([np.inf, -np.inf], '')

# 8) Build the list of lists, mapping '' → None so Sheets sees true blanks
headers = data_to_update.columns.tolist()
rows    = data_to_update.values.tolist()
values  = [headers]
for row in rows:
    clean = [None if (isinstance(c, str) and c=='') else c for c in row]
    values.append(clean)

# 9) Update the sheet at B5 (will expand to the right and down)
worksheet.update('B5', values)


/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_86043/2774254129.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_to_update = data_to_update.applymap(lambda x: x.strip() if isinstance(x, str) else x)
/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_86043/2774254129.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_to_update = data_to_update.applymap(
/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_86043/2774254129.py:33: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('B5', values)


{'spreadsheetId': '1A6Q8dvoWwLi26tnQQoEo6ZnekvZBQbpgCyP20iQV6Ug',
 'updatedRange': 'SQLs!B5:J57',
 'updatedRows': 53,
 'updatedColumns': 9,
 'updatedCells': 469}

In [9]:
# --- 2) SLIMDF FOR SAO PIVOT ---
df_sao = df[[
    'SAO Date',
    'SAO Date_Quarter',
    'Source',
    'Segment - historical',
    'Bookings Type'
]].copy()

# Only keep rows that actually have an SAO date
df_sao = df_sao[df_sao['SAO Date'].notna()]

# Map “Other” sources
allowed_src = ['AE','BDR','Channel','Marketing','Success']
df_sao['Source'] = df_sao['Source'].where(df_sao['Source'].isin(allowed_src), 'Other')

# Filter segments & types
allowed_segs  = ['Enterprise','Mid Market','SMB']
allowed_types = ['Expansion','New Business']
df_sao = df_sao[
    df_sao['Segment - historical'].isin(allowed_segs) &
    df_sao['Bookings Type'].isin(allowed_types)
]

# --- 3) STATIC ROW LAYOUT (same as before) ---
rows = []
for src in ['AE','BDR','Channel','Marketing']:
    for seg in allowed_segs:
        for bt in allowed_types:
            rows.append((src, seg, bt))
        rows.append((src, seg, f'{seg} Total'))
    rows.append((src, '', f'{src} Total'))

rows.append(('Other','','Other Total'))

for seg in allowed_segs:
    for bt in allowed_types:
        rows.append(('Success', seg, bt))
    rows.append(('Success', seg, f'{seg} Total'))
rows.append(('Success','','Success Total'))

rows.append(('','','Grand Total'))

index = pd.MultiIndex.from_tuples(
    rows,
    names=['Source','Segment - historical','Bookings Type']
)

# --- 4) INIT PIVOT SHELL ---
pivot_sao = pd.DataFrame(
    0,
    index=index,
    columns=quarters_5 + ['Grand Total']
)

# --- 5) DETAIL COUNTS BY SAO Date Quarter ---
grp = df_sao.groupby(
    ['Source','Segment - historical','Bookings Type','SAO Date_Quarter']
).size()

for (src, seg, bt, q), cnt in grp.items():
    if (src,seg,bt) in pivot_sao.index and q in quarters_5:
        pivot_sao.at[(src,seg,bt), q] = cnt

# --- 6) SEGMENT SUBTOTALS ---
grp_seg = df_sao.groupby(
    ['Source','Segment - historical','SAO Date_Quarter']
).size()

for (src, seg, q), cnt in grp_seg.items():
    row = (src, seg, f'{seg} Total')
    if row in pivot_sao.index and q in quarters_5:
        pivot_sao.at[row, q] = cnt

# --- 7) SOURCE SUBTOTALS ---
grp_src = df_sao.groupby(['Source','SAO Date_Quarter']).size()

for (src, q), cnt in grp_src.items():
    row = (src, '', f'{src} Total')
    if row in pivot_sao.index and q in quarters_5:
        pivot_sao.at[row, q] = cnt

# --- 8) GRAND TOTAL PER QUARTER ---
grp_all = df_sao.groupby('SAO Date_Quarter').size()
for q, cnt in grp_all.items():
    if q in quarters_5:
        pivot_sao.at[('', '', 'Grand Total'), q] = cnt

# --- 9) FINAL GRAND TOTAL COLUMN ---
pivot_sao['Grand Total'] = pivot_sao[quarters_5].sum(axis=1)

# --- 10) RESET FOR EXPORT ---
pivot_sao_display = pivot_sao.reset_index()

# --- 11) PUSH TO GOOGLE SHEETS at B5 in "SAOs" ---
worksheet_sao = sht1.worksheet("SAOs")

# Prepare values (map ''→None for true blanks)
data_to_update = pivot_sao_display.copy()
data_to_update.columns = data_to_update.columns.astype(str)
data_to_update = data_to_update.applymap(lambda x: x.strip() if isinstance(x,str) else x)
data_to_update = data_to_update.replace(r'^\s*$', '', regex=True)
data_to_update = data_to_update.fillna('').replace([np.inf, -np.inf], '')
headers = data_to_update.columns.tolist()
rows = data_to_update.values.tolist()
values = [headers]
for row in rows:
    values.append([None if (isinstance(c,str) and c=='') else c for c in row])

worksheet_sao.update('B5', values)

/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_86043/2049992231.py:101: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_to_update = data_to_update.applymap(lambda x: x.strip() if isinstance(x,str) else x)
/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_86043/2049992231.py:110: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet_sao.update('B5', values)


{'spreadsheetId': '1A6Q8dvoWwLi26tnQQoEo6ZnekvZBQbpgCyP20iQV6Ug',
 'updatedRange': 'SAOs!B5:J57',
 'updatedRows': 53,
 'updatedColumns': 9,
 'updatedCells': 469}

In [10]:
# --- 2) SLIMDF FOR SAO PIPGEN ---
df_pip = df[[
    'SAO Date_Quarter',
    'Source',
    'Segment - historical',
    'Bookings Type',
    'ARR Change'
]].copy()

# Only keep rows with an SAO
df_pip = df_pip[df_pip['SAO Date_Quarter'].notna()]

# Map non-standard sources into “Other”
allowed_src = ['AE','BDR','Channel','Marketing','Success']
df_pip['Source'] = df_pip['Source'].where(
    df_pip['Source'].isin(allowed_src), 'Other'
)

# Filter segments & booking types
allowed_segs  = ['Enterprise','Mid Market','SMB']
allowed_types = ['Expansion','New Business']
df_pip = df_pip[
    df_pip['Segment - historical'].isin(allowed_segs) &
    df_pip['Bookings Type'].isin(allowed_types)
]

# --- 3) STATIC ROW INDEX (same as before) ---
rows = []
for src in ['AE','BDR','Channel','Marketing']:
    for seg in allowed_segs:
        for bt in allowed_types:
            rows.append((src, seg, bt))
        rows.append((src, seg, f'{seg} Total'))
    rows.append((src, '', f'{src} Total'))

rows.append(('Other','','Other Total'))

for seg in allowed_segs:
    for bt in allowed_types:
        rows.append(('Success', seg, bt))
    rows.append(('Success', seg, f'{seg} Total'))
rows.append(('Success','','Success Total'))

rows.append(('','','Grand Total'))

index = pd.MultiIndex.from_tuples(
    rows,
    names=['Source','Segment - historical','Bookings Type']
)

# --- 4) INIT PIVOT SHELL ---
pivot_pip = pd.DataFrame(
    0.0,
    index=index,
    columns=quarters_5 + ['Grand Total']
)

# --- 5) DETAIL: sum ARR Change by SAO Date_Quarter ---
grp = df_pip.groupby(
    ['Source','Segment - historical','Bookings Type','SAO Date_Quarter']
)['ARR Change'].sum()

for (src, seg, bt, q), total in grp.items():
    if (src,seg,bt) in pivot_pip.index and q in quarters_5:
        pivot_pip.at[(src,seg,bt), q] = total

# --- 6) SEGMENT SUBTOTALS (sum ARR Change) ---
grp_seg = df_pip.groupby(
    ['Source','Segment - historical','SAO Date_Quarter']
)['ARR Change'].sum()

for (src, seg, q), total in grp_seg.items():
    row = (src, seg, f'{seg} Total')
    if row in pivot_pip.index and q in quarters_5:
        pivot_pip.at[row, q] = total

# --- 7) SOURCE SUBTOTALS ---
grp_src = df_pip.groupby(['Source','SAO Date_Quarter'])['ARR Change'].sum()
for (src, q), total in grp_src.items():
    row = (src, '', f'{src} Total')
    if row in pivot_pip.index and q in quarters_5:
        pivot_pip.at[row, q] = total

# --- 8) GRAND TOTAL PER QUARTER ---
grp_all = df_pip.groupby('SAO Date_Quarter')['ARR Change'].sum()
for q, total in grp_all.items():
    if q in quarters_5:
        pivot_pip.at[('', '', 'Grand Total'), q] = total

# --- 9) GRAND TOTAL COLUMN ---
pivot_pip['Grand Total'] = pivot_pip[quarters_5].sum(axis=1)

# --- 10) RESET INDEX FOR EXPORT ---
pivot_pip_display = pivot_pip.reset_index()

# --- 11) PUSH INTO GOOGLE SHEETS AT B5 on sheet "SAO_Pipgen" ---
worksheet_pip = sht1.worksheet("Pipegen")

# Clean & prepare values
data_to_update = pivot_pip_display.copy()
data_to_update.columns = data_to_update.columns.astype(str)
data_to_update = data_to_update.applymap(lambda x: x.strip() if isinstance(x,str) else x)
data_to_update = data_to_update.replace(r'^\s*$', '', regex=True)
data_to_update = data_to_update.fillna('').replace([np.inf, -np.inf], '')
headers = data_to_update.columns.tolist()
rows    = data_to_update.values.tolist()
values  = [headers]
for row in rows:
    values.append([None if (isinstance(c,str) and c=='') else c for c in row])

worksheet_pip.update('B5', values)

/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_86043/2869846744.py:102: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_to_update = data_to_update.applymap(lambda x: x.strip() if isinstance(x,str) else x)
/var/folders/89/mmk3x71s01g09b0bf8137yrw0000gp/T/ipykernel_86043/2869846744.py:111: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet_pip.update('B5', values)


{'spreadsheetId': '1A6Q8dvoWwLi26tnQQoEo6ZnekvZBQbpgCyP20iQV6Ug',
 'updatedRange': 'Pipegen!B5:J57',
 'updatedRows': 53,
 'updatedColumns': 9,
 'updatedCells': 469}

In [11]:
BreakBreakBreak production

SyntaxError: invalid syntax (1412344556.py, line 1)

In [ ]:
# --- Assumes you already have:
# df, df_pipegen, get_quarter_info(), get_last_completed_quarters(), etc.

today = datetime.today()
active_quarter = get_quarter_info(today)[0]
last_4 = get_last_completed_quarters(4, today)
quarters_5 = last_4 + [active_quarter]

# 1) BOOKINGS METRICS
bk = df[df['Stage']=='Closed Won'].copy()
bk = bk[bk['Close Date_Quarter'].isin(quarters_5)]
bookings_summary = (
    bk.groupby(['Close Date_Quarter','Segment - historical'])
      .agg(
        bookings_arr=('ARR Change','sum'),
        deals_closed=('SFDC ID 18 Digit','nunique'),
        avg_deal_size=('ARR Change','mean')
      )
      .rename_axis(['Quarter','Segment'])
      .reset_index()
)
bookings_wide = bookings_summary.pivot(
    index='Segment', columns='Quarter',
    values=['bookings_arr','deals_closed','avg_deal_size']
).fillna(0)
bookings_wide.columns = ['_'.join(col) for col in bookings_wide.columns]
bookings_wide.reset_index(inplace=True)

# 2) PIPEGEN METRICS
pg = df[df['SAO Date_Quarter'].notna()].copy()
pg = pg[pg['SAO Date_Quarter'].isin(quarters_5)]
pipegen_summary = (
    pg.groupby(['SAO Date_Quarter','Segment - historical'])
      .agg(pipegen_arr=('ARR Change','sum'))
      .rename_axis(['Quarter','Segment'])
      .reset_index()
)
pipegen_wide = pipegen_summary.pivot(
    index='Segment', columns='Quarter', values='pipegen_arr'
).fillna(0)
pipegen_wide.columns = [f'pipegen_arr_{q}' for q in pipegen_wide.columns]
pipegen_wide.reset_index(inplace=True)

# 3) COMBINE
master = bookings_wide.merge(pipegen_wide, on='Segment', how='outer').fillna(0)

# 4) SAFER QoQ & YoY FUNCTION
def add_pct_changes(df, metric_prefix, quarters):
    # Quarter-over-Quarter
    for i in range(1, len(quarters)):
        prev, curr = quarters[i-1], quarters[i]
        c_prev, c_curr = f"{metric_prefix}_{prev}", f"{metric_prefix}_{curr}"
        if c_prev in df.columns and c_curr in df.columns:
            df[f"{metric_prefix}_QoQ_{prev}_to_{curr}"] = (
                (df[c_curr] - df[c_prev]) / df[c_prev].replace(0, np.nan)
            ) * 100
    # Year-over-Year (compare first vs last)
    yoy_prev, yoy_curr = quarters[0], quarters[-1]
    c_prev, c_curr = f"{metric_prefix}_{yoy_prev}", f"{metric_prefix}_{yoy_curr}"
    if c_prev in df.columns and c_curr in df.columns:
        df[f"{metric_prefix}_YoY_{yoy_prev}_to_{yoy_curr}"] = (
            (df[c_curr] - df[c_prev]) / df[c_prev].replace(0, np.nan)
        ) * 100

# 5) APPLY CHANGES
add_pct_changes(master, "bookings_arr", quarters_5)
add_pct_changes(master, "pipegen_arr", quarters_5)

In [ ]:
def get_last_completed_quarters(n, today=None):
    """
    Returns a list of the last n completed quarters (as strings) based on the custom fiscal calendar.
    For example, if today is 03/06/2025 (current quarter = "2026-Q1"), the most recent completed quarter
    is "2025-Q4".
    """
    if today is None:
        today = datetime.today()
    # Determine the current quarter (which is in progress)
    current_quarter, q_start, q_end = get_quarter_info(today)
    # Last completed quarter: one day before current quarter starts.
    last_date = q_start - timedelta(days=1)
    quarters = []
    for _ in range(n):
        q, q_start, q_end = get_quarter_info(last_date)
        quarters.append(q)
        last_date = q_start - timedelta(days=1)
    # Reverse to show them in chronological order (oldest left, newest right)
    return quarters[::-1]

# ---------------------------
# Define dynamic parameters and filter the data
# ---------------------------
# Set "today" dynamically
today = datetime.today()
last_5_quarters = get_last_completed_quarters(5, today)
print("Last 5 completed quarters:", last_5_quarters)

# Define the three valid segments
valid_segments = ['Enterprise', 'Mid Market', 'SMB']

# Filter the master DataFrame for the segments of interest
df_filtered = df[df['Segment - historical'].isin(valid_segments)]

# For bookings, we only want opportunities with Stage = "Closed Won"
df_bookings = df_filtered[df_filtered['Stage'] == 'Closed Won']

# Ensure that the "Close Date_Quarter" column exists.
# If it doesn’t, create it using get_quarter_info on the "Close Date" column.
if 'Close Date_Quarter' not in df_bookings.columns:
    def get_close_quarter(date_val):
        if pd.isnull(date_val):
            return None
        q, _, _ = get_quarter_info(date_val)
        return q
    df_bookings['Close Date_Quarter'] = df_bookings['Close Date'].apply(get_close_quarter)

# Now, filter df_bookings to only include rows from the last 5 completed quarters
df_bookings = df_bookings[df_bookings['Close Date_Quarter'].isin(last_5_quarters)]

# ---------------------------
# View 1: Bookings by Quarter (by Segment)
# ---------------------------
view1 = pd.pivot_table(
    df_bookings,
    index='Segment - historical',
    columns='Close Date_Quarter',
    values='ARR Change',
    aggfunc='sum',
    fill_value=0
)
# Reorder columns to match last_5_quarters
view1 = view1.reindex(columns=last_5_quarters)
# Add a "Total" row that sums each quarter
view1.loc['Total'] = view1.sum()

# ---------------------------
# View 2: Bookings Type Breakdown (Gross and %)
# ---------------------------
# Pivot table with rows = Bookings Type, columns = Close Date_Quarter, values = sum of ARR Change
view2_gross = pd.pivot_table(
    df_bookings,
    index='Bookings Type',
    columns='Close Date_Quarter',
    values='ARR Change',
    aggfunc='sum',
    fill_value=0
)
view2_gross = view2_gross.reindex(columns=last_5_quarters)

# Create a percentage view (each quarter column sums to 100)
view2_pct = view2_gross.div(view2_gross.sum(axis=0), axis=1) * 100
view2_pct = view2_pct.round(2)

# ---------------------------
# View 3: Bookings Type by Segment (Gross and %)
# ---------------------------
# Pivot table with a multi-index: rows = [Segment, Bookings Type], columns = Close Date_Quarter
view3_gross = pd.pivot_table(
    df_bookings,
    index=['Segment - historical', 'Bookings Type'],
    columns='Close Date_Quarter',
    values='ARR Change',
    aggfunc='sum',
    fill_value=0
)
view3_gross = view3_gross.reindex(columns=last_5_quarters)

# Create a percentage view.
# For each segment, for each quarter column, calculate the percentage breakdown by Bookings Type.
view3_pct = view3_gross.copy()
for seg in view3_pct.index.get_level_values(0).unique():
    idx = view3_pct.index.get_level_values(0) == seg
    view3_pct.loc[idx] = view3_pct.loc[idx].div(view3_pct.loc[idx].sum(), axis=1) * 100
view3_pct = view3_pct.round(2)

In [ ]:
view1

In [ ]:
view2_gross

In [ ]:
view2_pct

In [ ]:
view3_gross

In [ ]:
view3_pct

In [ ]:
# ---------------------------
# Data Preparation
# ---------------------------
# Assume your SFDC report is in DataFrame "df" with date fields processed.
# Filter for bookings (Stage = "Closed Won")
df_bookings = df[df['Stage'] == 'Closed Won'].copy()
df_bookings[['Day_of_Quarter','Total_Days_in_Quarter','Pct_Day']] = df_bookings['Close Date'].apply(compute_day_of_quarter)
df_bookings['Pct_Day_Bin'] = df_bookings['Pct_Day'].round(0)
df_bookings['Quarter'] = df_bookings['Close Date'].apply(lambda x: get_quarter_info(x)[0] if pd.notnull(x) else None)

today = datetime.today()
active_quarter, active_q_start, active_q_end = get_quarter_info(today)
today_pct_bin = round(compute_day_of_quarter(today)["Pct_Day"])

# For same quarter comparisons, define quarters for past 2 years
year_current = int(active_quarter.split('-')[0])
q_part = active_quarter.split('-')[1]
quarter_last_year = f"{year_current - 1}-{q_part}"
quarter_two_years_ago = f"{year_current - 2}-{q_part}"

# Fixed color mapping for same-quarter comparisons (View 2 & 4)
fixed_colors = {
    quarter_two_years_ago: "#1f77b4",  # blue
    quarter_last_year: "#2ca02c",       # green
    active_quarter: "#d62728"           # red (current quarter)
}

# ---------------------------
# View 1: Company-wide – Current Quarter vs. Historical Average (Last 8 Completed Quarters)
# ---------------------------
last_8_quarters = get_last_completed_quarters(8, today)
view1_quarters = last_8_quarters + [active_quarter]
df_view1 = df_bookings[df_bookings['Quarter'].isin(view1_quarters)].copy()
cum_view1_raw = compute_cumulative_raw(df_view1)
hist_cols = [q for q in last_8_quarters if q in cum_view1_raw.columns]
current_col = active_quarter if active_quarter in cum_view1_raw.columns else None

historical_avg_curve = cum_view1_raw[hist_cols].mean(axis=1)
target_value_view1 = historical_avg_curve.loc[100]
historical_avg_pacing = (historical_avg_curve / target_value_view1) * 100
if current_col is not None:
    current_cum = cum_view1_raw[current_col]
    current_pacing = (current_cum / target_value_view1) * 100
else:
    current_pacing = pd.Series([np.nan]*101, index=range(0,101))
current_pacing_truncated = current_pacing[current_pacing.index <= today_pct_bin]

df_view1_pacing = pd.DataFrame({
    'Pct_Day_Bin': range(0, 101),
    'Historical Avg Pacing (%)': historical_avg_pacing,
    'Current Quarter Pacing (%)': current_pacing
}).set_index('Pct_Day_Bin')

fig_view1 = go.Figure()
fig_view1.add_trace(go.Scatter(x=historical_avg_pacing.index, y=historical_avg_pacing.values,
                               mode='lines+markers', name='Historical Avg',
                               line=dict(color="#1f77b4")))  # blue
fig_view1.add_trace(go.Scatter(x=current_pacing_truncated.index, y=current_pacing_truncated.values,
                               mode='lines+markers', name='Current Quarter',
                               line=dict(color="#d62728")))  # red
fig_view1.update_layout(title='Company-wide Bookings Pacing: Current Quarter vs. Historical Average (Last 8 Quarters)',
                        xaxis_title='% of Quarter Completed',
                        yaxis_title='Pacing',
                        yaxis_range=[0, 105],
                        autosize=False, width=1600, height=960)
fig_view1.show()

print("View 1 Data (Company-wide):")
print(df_view1_pacing)

# ---------------------------
# View 2: Company-wide – Current Quarter vs. Same Quarter (Past 2 Years)
# ---------------------------
view2_quarters = [quarter_two_years_ago, quarter_last_year, active_quarter]
df_view2 = df_bookings[df_bookings['Quarter'].isin(view2_quarters)].copy()
cum_view2_raw = compute_cumulative_raw(df_view2)
df_view2_pct = pd.DataFrame(index=range(0,101))
for q in view2_quarters:
    if q not in cum_view2_raw.columns:
        continue
    if q == active_quarter:
        if (quarter_last_year in cum_view2_raw.columns) and (quarter_two_years_ago in cum_view2_raw.columns):
            target_active = np.mean([cum_view2_raw[quarter_last_year].iloc[100],
                                      cum_view2_raw[quarter_two_years_ago].iloc[100]])
        else:
            target_active = np.nan
        series = cum_view2_raw[q].copy()
        pacing_series = series.apply(lambda x: (x / target_active)*100 if pd.notnull(x) and target_active > 0 else np.nan)
        pacing_current = pacing_series[pacing_series.index <= today_pct_bin]
        df_view2_pct[q] = pacing_current
    else:
        target_q = cum_view2_raw[q].iloc[100]
        series = cum_view2_raw[q].copy()
        pacing_series = series.apply(lambda x: (x / target_q)*100 if pd.notnull(x) and target_q > 0 else np.nan)
        df_view2_pct[q] = pacing_series

fig_view2 = go.Figure()
for q in view2_quarters:
    if q not in df_view2_pct.columns:
        continue
    series = df_view2_pct[q].dropna()
    fig_view2.add_trace(go.Scatter(x=series.index, y=series.values,
                                   mode='lines+markers', name=f'{q}',
                                   line=dict(color=fixed_colors.get(q, "#000000"))))
fig_view2.update_layout(title='Company-wide Bookings Pacing: Current Quarter vs. Same Quarter (Past 2 Years)',
                        xaxis_title='% of Quarter Completed',
                        yaxis_title='Pacing',
                        yaxis_range=[0, 105],
                        autosize=False, width=1600, height=960)
fig_view2.show()

print("View 2 Data (Company-wide):")
print(df_view2_pct)

# ---------------------------
# View 3: Segment-Specific – Current Quarter vs. Historical Average (Last 8 Completed Quarters)
# ---------------------------
segments = ['SMB', 'Mid Market', 'Enterprise']
for seg in segments:
    df_seg = df_bookings[df_bookings['Segment - historical'] == seg].copy()
    if df_seg.empty:
        print(f"No data for segment: {seg}")
        continue
    
    df_seg_view1 = df_seg[df_seg['Quarter'].isin(view1_quarters)].copy()
    cum_seg_raw = compute_cumulative_raw(df_seg_view1)
    
    seg_hist_cols = [q for q in last_8_quarters if q in cum_seg_raw.columns]
    seg_current_col = active_quarter if active_quarter in cum_seg_raw.columns else None
    if len(seg_hist_cols) == 0 or seg_current_col is None:
        print(f"Insufficient data for segment: {seg}")
        continue
    
    seg_historical_avg_curve = cum_seg_raw[seg_hist_cols].mean(axis=1)
    seg_target_value = seg_historical_avg_curve.loc[100]
    seg_historical_avg_pacing = (seg_historical_avg_curve / seg_target_value) * 100
    seg_current_pacing = (cum_seg_raw[active_quarter] / seg_target_value) * 100
    seg_current_pacing_truncated = seg_current_pacing[seg_current_pacing.index <= today_pct_bin]
    
    fig_seg = go.Figure()
    fig_seg.add_trace(go.Scatter(x=seg_historical_avg_pacing.index,
                                 y=seg_historical_avg_pacing.values,
                                 mode='lines+markers',
                                 name='Historical Avg',
                                 line=dict(color="#1f77b4")))
    fig_seg.add_trace(go.Scatter(x=seg_current_pacing_truncated.index,
                                 y=seg_current_pacing_truncated.values,
                                 mode='lines+markers',
                                 name='Current Quarter',
                                 line=dict(color="#d62728")))
    fig_seg.update_layout(title=f'{seg} Bookings Pacing: Current Quarter vs. Historical Average (Last 8 Quarters)',
                          xaxis_title='% of Quarter Completed',
                          yaxis_title='Pacing',
                          yaxis_range=[0,105],
                          autosize=False, width=1600, height=960)
    fig_seg.show()
    file_path = os.path.join('master', f'bookings_{seg}_view3.png')
    fig_seg.write_image(file_path)
    
    seg_pacing_df = pd.DataFrame({
        'Historical Avg Pacing (%)': seg_historical_avg_pacing,
        'Current Quarter Pacing (%)': seg_current_pacing
    }).set_index(seg_historical_avg_pacing.index)
    print(f"{seg} Historical vs. Current Pacing Data:")
    print(seg_pacing_df)

# ---------------------------
# View 4: Segment-Specific – Current vs. Same Quarter (Past 2 Years)
# ---------------------------
for seg in ['SMB', 'Mid Market', 'Enterprise']:
    df_seg = df_bookings[df_bookings['Segment - historical'] == seg].copy()
    if df_seg.empty:
        print(f"No data for segment: {seg}")
        continue
    df_seg_view2 = df_seg[df_seg['Quarter'].isin(view2_quarters)].copy()
    cum_seg_view2_raw = compute_cumulative_raw(df_seg_view2)
    df_seg_view2_pct = pd.DataFrame(index=range(0,101))
    for q in view2_quarters:
        if q not in cum_seg_view2_raw.columns:
            continue
        if q == active_quarter:
            if (quarter_last_year in cum_seg_view2_raw.columns) and (quarter_two_years_ago in cum_seg_view2_raw.columns):
                target_active = np.mean([cum_seg_view2_raw[quarter_last_year].iloc[100],
                                          cum_seg_view2_raw[quarter_two_years_ago].iloc[100]])
            else:
                target_active = np.nan
            series = cum_seg_view2_raw[q].copy()
            pacing_series = series.apply(lambda x: (x / target_active)*100 if pd.notnull(x) and target_active > 0 else np.nan)
            pacing_current = pacing_series[pacing_series.index <= today_pct_bin]
            df_seg_view2_pct[q] = pacing_current
        else:
            target_q = cum_seg_view2_raw[q].iloc[100]
            series = cum_seg_view2_raw[q].copy()
            pacing_series = series.apply(lambda x: (x / target_q)*100 if pd.notnull(x) and target_q > 0 else np.nan)
            df_seg_view2_pct[q] = pacing_series
    fig_seg2 = go.Figure()
    for q in view2_quarters:
        if q not in df_seg_view2_pct.columns:
            continue
        series = df_seg_view2_pct[q].dropna()
        fig_seg2.add_trace(go.Scatter(
            x=series.index,
            y=series.values,
            mode='lines+markers',
            name=f'{q}',
            line=dict(color=fixed_colors.get(q, "#000000"))
        ))
    fig_seg2.update_layout(title=f'{seg} Bookings Pacing: Current vs. Same Quarter (Past 2 Years)',
                           xaxis_title='% of Quarter Completed',
                           yaxis_title='Pacing',
                           yaxis_range=[0,105],
                           autosize=False, width=1600, height=960)
    fig_seg2.show()
    file_path = os.path.join('master', f'bookings_{seg}_view4.png')
    fig_seg2.write_image(file_path)
    print(f"Segment {seg} Pacing Data (Same Quarter Comparison):")
    print(df_seg_view2_pct)

In [ ]:
# ---------------------------
# Preprocess Pipegen Data (based on SAO Date)
# ---------------------------
df_pipegen = df[df['SAO Date'].notnull()].copy()
df_pipegen['SAO Date'] = pd.to_datetime(df_pipegen['SAO Date'], errors='coerce')
df_pipegen[['Pipegen_Day_of_Quarter','Pipegen_Total_Days_in_Quarter','Pipegen_Pct_Day']] = df_pipegen['SAO Date'].apply(compute_day_of_quarter)
df_pipegen['Pipegen_Pct_Day_Bin'] = df_pipegen['Pipegen_Pct_Day'].round(0)
df_pipegen['Pipegen_Quarter'] = df_pipegen['SAO Date'].apply(lambda x: get_quarter_info(x)[0] if pd.notnull(x) else None)

today = datetime.today()
active_pipegen_quarter, active_pipegen_q_start, active_pipegen_q_end = get_quarter_info(today)
pipegen_today_pct_bin = round(compute_day_of_quarter(today)["Pct_Day"])

# Define last 8 completed quarters and same-quarter comparisons for Pipegen
last_8_quarters_pipegen = get_last_completed_quarters(8, today)
view1_quarters_pipegen = last_8_quarters_pipegen + [active_pipegen_quarter]

year_current = int(active_pipegen_quarter.split('-')[0])
q_part = active_pipegen_quarter.split('-')[1]
quarter_last_year = f"{year_current - 1}-{q_part}"
quarter_two_years_ago = f"{year_current - 2}-{q_part}"
view2_quarters_pipegen = [quarter_two_years_ago, quarter_last_year, active_pipegen_quarter]

# ---------------------------
# Remove unwanted Sources ("Other" and "Connect")
# ---------------------------
all_sources_pipegen = df_pipegen['Source'].unique()
sources_pipegen = [s for s in all_sources_pipegen if s not in ['Other', 'Connect']]

# ---------------------------
# Fixed color mapping for same-quarter comparisons (View 2 & 4)
# ---------------------------
fixed_colors = {
    quarter_two_years_ago: "#1f77b4",  # blue
    quarter_last_year: "#2ca02c",       # green
    active_pipegen_quarter: "#d62728"   # red (current quarter)
}

# ---------------------------
# Ensure the 'master' folder exists for saving charts
# ---------------------------
CHARTS_DIR = 'master'
os.makedirs(CHARTS_DIR, exist_ok=True)

# ===========================
# PIPEGEN VIEW 1: Company-wide – Current Quarter vs. Historical Average (Last 8 Quarters)
# (One chart per Source)
# ===========================
for src in sources_pipegen:
    df_src = df_pipegen[df_pipegen['Source'] == src].copy()
    df_src_view1 = df_src[df_src['Pipegen_Quarter'].isin(view1_quarters_pipegen)].copy()
    cum_src_raw = compute_cumulative_raw_pipegen(df_src_view1)
    hist_cols_src = [q for q in last_8_quarters_pipegen if q in cum_src_raw.columns]
    current_col_src = active_pipegen_quarter if active_pipegen_quarter in cum_src_raw.columns else None
    if len(hist_cols_src) == 0 or current_col_src is None:
        continue
    historical_avg_curve_src = cum_src_raw[hist_cols_src].mean(axis=1)
    target_value_src = historical_avg_curve_src.loc[100]
    historical_avg_pacing_src = (historical_avg_curve_src / target_value_src) * 100
    current_cum_src = cum_src_raw[current_col_src]
    current_pacing_src = (current_cum_src / target_value_src) * 100
    current_pacing_src_truncated = current_pacing_src[current_pacing_src.index <= pipegen_today_pct_bin]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
         x=historical_avg_pacing_src.index,
         y=historical_avg_pacing_src.values,
         mode='lines+markers',
         name=f'Historical Avg - {src}',
         line=dict(color="#1f77b4")
    ))
    fig.add_trace(go.Scatter(
         x=current_pacing_src_truncated.index,
         y=current_pacing_src_truncated.values,
         mode='lines+markers',
         name=f'Current Quarter - {src}',
         line=dict(color="#d62728")
    ))
    fig.update_layout(title=f'Pipegen: {src} – Current vs. Historical Average (Last 8 Quarters)',
                      xaxis_title='% of Quarter Completed',
                      yaxis_title='Pacing',
                      yaxis_range=[0,105],
                      autosize=False, width=1600, height=960)
    fig.show()
    file_path = os.path.join(CHARTS_DIR, f'pipegen_{src}_view1.png')
    fig.write_image(file_path)

# ===========================
# PIPEGEN VIEW 2: Company-wide – Current Quarter vs. Same Quarter (Past 2 Years)
# (One chart per Source; fixed colors used for the three quarters)
# ===========================
for src in sources_pipegen:
    df_src = df_pipegen[df_pipegen['Source'] == src].copy()
    df_src_view2 = df_src[df_src['Pipegen_Quarter'].isin(view2_quarters_pipegen)].copy()
    cum_src_raw = compute_cumulative_raw_pipegen(df_src_view2)
    df_src_pct = pd.DataFrame(index=range(0,101))
    for q in view2_quarters_pipegen:
        if q not in cum_src_raw.columns:
            continue
        if q == active_pipegen_quarter:
            if (quarter_last_year in cum_src_raw.columns) and (quarter_two_years_ago in cum_src_raw.columns):
                target_active = np.mean([cum_src_raw[quarter_last_year].iloc[100],
                                          cum_src_raw[quarter_two_years_ago].iloc[100]])
            else:
                target_active = np.nan
            series = cum_src_raw[q].copy()
            pacing_series = series.apply(lambda x: (x / target_active)*100 if pd.notnull(x) and target_active > 0 else np.nan)
            pacing_current = pacing_series[pacing_series.index <= pipegen_today_pct_bin]
            df_src_pct[q] = pacing_current
        else:
            target_q = cum_src_raw[q].iloc[100]
            series = cum_src_raw[q].copy()
            pacing_series = series.apply(lambda x: (x / target_q)*100 if pd.notnull(x) and target_q > 0 else np.nan)
            df_src_pct[q] = pacing_series
    fig = go.Figure()
    for q in view2_quarters_pipegen:
        if q not in df_src_pct.columns:
            continue
        series = df_src_pct[q].dropna()
        fig.add_trace(go.Scatter(
             x=series.index,
             y=series.values,
             mode='lines+markers',
             name=f'{q}',
             line=dict(color=fixed_colors.get(q, "#000000"))
        ))
    fig.update_layout(title=f'Pipegen: {src} – Current vs. Same Quarter (Past 2 Years)',
                      xaxis_title='% of Quarter Completed',
                      yaxis_title='Pacing',
                      yaxis_range=[0,105],
                      autosize=False, width=1600, height=960)
    fig.show()
    file_path = os.path.join(CHARTS_DIR, f'pipegen_{src}_view2.png')
    fig.write_image(file_path)

# ===========================
# PIPEGEN VIEW 3: Segment-Specific – Current Quarter vs. Historical Average (Last 8 Quarters)
# (One chart per Segment per Source)
# ===========================
for seg in ['SMB', 'Mid Market', 'Enterprise']:
    df_seg = df_pipegen[df_pipegen['Segment - historical'] == seg].copy()
    if df_seg.empty:
        print(f"No pipegen data for segment: {seg}")
        continue
    sources_seg = [s for s in df_seg['Source'].unique() if s not in ['Other', 'Connect']]
    for src in sources_seg:
        df_seg_src = df_seg[df_seg['Source'] == src].copy()
        df_seg_src_view1 = df_seg_src[df_seg_src['Pipegen_Quarter'].isin(view1_quarters_pipegen)].copy()
        cum_seg_src_raw = compute_cumulative_raw_pipegen(df_seg_src_view1)
        seg_hist_cols_src = [q for q in last_8_quarters_pipegen if q in cum_seg_src_raw.columns]
        seg_current_col_src = active_pipegen_quarter if active_pipegen_quarter in cum_seg_src_raw.columns else None
        if len(seg_hist_cols_src) == 0 or seg_current_col_src is None:
            continue
        seg_historical_avg_curve = cum_seg_src_raw[seg_hist_cols_src].mean(axis=1)
        seg_target_value = seg_historical_avg_curve.loc[100]
        seg_historical_avg_pacing = (seg_historical_avg_curve / seg_target_value) * 100
        seg_current_cum = cum_seg_src_raw[seg_current_col_src]
        seg_current_pacing = (seg_current_cum / seg_target_value) * 100
        seg_current_pacing_truncated = seg_current_pacing[seg_current_pacing.index <= pipegen_today_pct_bin]
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(
             x=seg_historical_avg_pacing.index,
             y=seg_historical_avg_pacing.values,
             mode='lines+markers',
             name=f'Historical Avg - {src}',
             line=dict(color="#1f77b4")
        ))
        fig.add_trace(go.Scatter(
             x=seg_current_pacing_truncated.index,
             y=seg_current_pacing_truncated.values,
             mode='lines+markers',
             name=f'Current Quarter - {src}',
             line=dict(color="#d62728")
        ))
        fig.update_layout(title=f'Pipegen {seg}: {src} – Current vs. Historical Average (Last 8 Quarters)',
                          xaxis_title='% of Quarter Completed',
                          yaxis_title='Pacing',
                          yaxis_range=[0,105],
                          autosize=False, width=1600, height=960)
        fig.show()
        file_path = os.path.join(CHARTS_DIR, f'pipegen_{seg}_{src}_view3.png')
        fig.write_image(file_path)

# ===========================
# PIPEGEN VIEW 4: Segment-Specific – Current Quarter vs. Same Quarter (Past 2 Years)
# (One chart per Segment per Source)
# ===========================
for seg in ['SMB', 'Mid Market', 'Enterprise']:
    df_seg = df_pipegen[df_pipegen['Segment - historical'] == seg].copy()
    if df_seg.empty:
        print(f"No pipegen data for segment: {seg}")
        continue
    sources_seg = [s for s in df_seg['Source'].unique() if s not in ['Other', 'Connect']]
    for src in sources_seg:
        df_seg_src = df_seg[df_seg['Source'] == src].copy()
        df_seg_src_view2 = df_seg_src[df_seg_src['Pipegen_Quarter'].isin(view2_quarters_pipegen)].copy()
        cum_seg_src_raw = compute_cumulative_raw_pipegen(df_seg_src_view2)
        df_seg_src_pct = pd.DataFrame(index=range(0,101))
        for q in view2_quarters_pipegen:
            if q not in cum_seg_src_raw.columns:
                continue
            if q == active_pipegen_quarter:
                if (quarter_last_year in cum_seg_src_raw.columns) and (quarter_two_years_ago in cum_seg_src_raw.columns):
                    target_active_seg = np.mean([cum_seg_src_raw[quarter_last_year].iloc[100],
                                                 cum_seg_src_raw[quarter_two_years_ago].iloc[100]])
                else:
                    target_active_seg = np.nan
                series = cum_seg_src_raw[q].copy()
                pacing_series = series.apply(lambda x: (x / target_active_seg)*100 if pd.notnull(x) and target_active_seg > 0 else np.nan)
                pacing_current = pacing_series[pacing_series.index <= pipegen_today_pct_bin]
                df_seg_src_pct[q] = pacing_current
            else:
                target_q = cum_seg_src_raw[q].iloc[100]
                series = cum_seg_src_raw[q].copy()
                pacing_series = series.apply(lambda x: (x / target_q)*100 if pd.notnull(x) and target_q > 0 else np.nan)
                df_seg_src_pct[q] = pacing_series
        fig = go.Figure()
        for q in view2_quarters_pipegen:
            if q not in df_seg_src_pct.columns:
                continue
            series = df_seg_src_pct[q].dropna()
            fig.add_trace(go.Scatter(
                x=series.index,
                y=series.values,
                mode='lines+markers',
                name=f'{q}',
                line=dict(color=fixed_colors.get(q, "#000000"))
            ))
        fig.update_layout(title=f'Pipegen {seg}: {src} – Current vs. Same Quarter (Past 2 Years)',
                          xaxis_title='% of Quarter Completed',
                          yaxis_title='Pacing',
                          yaxis_range=[0,105],
                          autosize=False, width=1600, height=960)
        fig.show()
        file_path = os.path.join(CHARTS_DIR, f'pipegen_{seg}_{src}_view4.png')
        fig.write_image(file_path)

In [ ]:
# ---- Assumptions:
# df_pipegen is your preprocessed DataFrame for Pipegen (based on SAO Date)
# It contains at least: 'Pipegen_Quarter', 'Source', and 'ARR Change'
# active_pipegen_quarter is defined (e.g., via get_quarter_info(today)[0])
# get_last_completed_quarters(n, today) is available.

today = datetime.today()

# Define last 8 completed quarters and include the current quarter
last_8_quarters_pipegen = get_last_completed_quarters(8, today)
quarters_to_include = last_8_quarters_pipegen + [active_pipegen_quarter]

# Filter the Pipegen DataFrame for these quarters
df_pipegen_filtered = df_pipegen[df_pipegen['Pipegen_Quarter'].isin(quarters_to_include)].copy()

# Group by Pipegen_Quarter and Source, summing ARR Change
grouped = df_pipegen_filtered.groupby(['Pipegen_Quarter', 'Source'])['ARR Change'].sum().reset_index()

# Pivot so that rows = Pipegen_Quarter and columns = Source
pivot = grouped.pivot(index='Pipegen_Quarter', columns='Source', values='ARR Change').fillna(0)

# Enforce our desired chronological order using quarters_to_include
pivot = pivot.reindex(quarters_to_include)

# Pre-calculate the total ARR change per quarter for the absolute chart
pivot_totals = pivot.sum(axis=1)

# ---------------------------
# Absolute Stacked Bar Chart (ARR Change in absolute terms)
# ---------------------------
fig_absolute = go.Figure()
# For each source, create labels that are shown only if:
#   - the value is >= 1,000 AND
#   - that value is at least 5% of the quarter's total
for source in pivot.columns:
    labels = []
    for q in pivot.index:
        v = pivot.at[q, source]
        total = pivot_totals.loc[q]
        # Avoid division by zero:
        if total == 0 or v < 1000 or (v / total * 100) < 5:
            labels.append("")
        else:
            labels.append(f'{v/1000:.1f}K')
    fig_absolute.add_trace(go.Bar(
         x=pivot.index,
         y=pivot[source],
         name=source,
         text=labels,
         textposition='auto',
         textfont=dict(size=14)
    ))
fig_absolute.update_layout(
    barmode='stack',
    title='Pipegen Contributions by Source (Absolute ARR Change)',
    xaxis_title='Quarter',
    yaxis_title='ARR Change',
    width=1600,
    height=960
)
fig_absolute.show()

# ---------------------------
# Normalized (100% Stacked) Bar Chart (Percentage contributions)
# ---------------------------
# Compute percentage contributions per quarter
pivot_sum = pivot.sum(axis=1).replace(0, 1)  # avoid division by zero
pivot_percent = pivot.div(pivot_sum, axis=0) * 100

# Set threshold for showing data labels in normalized chart (only show if >= 5%)
label_threshold = 2

fig_normalized = go.Figure()
for source in pivot_percent.columns:
    labels = [f'{val:.1f}%' if val >= label_threshold else '' for val in pivot_percent[source].values]
    fig_normalized.add_trace(go.Bar(
         x=pivot_percent.index,
         y=pivot_percent[source],
         name=source,
         text=labels,
         textposition='inside',
         textfont=dict(size=14)
    ))
fig_normalized.update_layout(
    barmode='stack',
    title='Pipegen Contributions by Source (Normalized to 100%)',
    xaxis_title='Quarter',
    yaxis_title='Percentage',
    yaxis=dict(range=[0, 100], tickvals=list(range(0, 101, 10))),
    width=1600,
    height=960
)
fig_normalized.show()

In [ ]:


# ---------------------------
# SAVE AGGREGATED PIPEGEN STACKED BAR CHARTS
# ---------------------------
abs_path_stack = os.path.join(CHARTS_DIR, 'pipegen_stacked_absolute.png')
fig_absolute.write_image(abs_path_stack)

norm_path_stack = os.path.join(CHARTS_DIR, 'pipegen_stacked_normalized.png')
fig_normalized.write_image(norm_path_stack)

print("All charts have been saved to the 'master' folder.")


In [ ]:
# ---------------------------
# CONFIGURATION
# ---------------------------
SPREADSHEET_ID = '1A6Q8dvoWwLi26tnQQoEo6ZnekvZBQbpgCyP20iQV6Ug'  # Replace with your spreadsheet ID
SHEET_NAME = 'Charts'  # The sheet name where you want to put the charts
CREDENTIALS_FILE = '/Users/bchen/Downloads/gspread-428120-06947c66447d.json'

SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# Set the directory for charts to 'master'
CHARTS_DIR = 'master'
os.makedirs(CHARTS_DIR, exist_ok=True)
# Get all PNG chart files from CHARTS_DIR
chart_files = [os.path.join(CHARTS_DIR, f) for f in os.listdir(CHARTS_DIR) if f.lower().endswith('.png')]

# ---------------------------
# AUTHENTICATION
# ---------------------------
creds = Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=SCOPES)
client = gspread.authorize(creds)
drive_service = build('drive', 'v3', credentials=creds)
sheet = client.open_by_key(SPREADSHEET_ID).worksheet(SHEET_NAME)

# ---------------------------
# DRIVE UPLOAD FUNCTION
# ---------------------------
def upload_image_to_drive(local_path):
    """
    Uploads a PNG image to Google Drive, makes it public, and returns its URL.
    """
    file_name = os.path.basename(local_path)
    file_metadata = {'name': file_name, 'mimeType': 'image/png'}
    media = MediaFileUpload(local_path, mimetype='image/png')
    uploaded_file = drive_service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()
    file_id = uploaded_file.get('id')
    # Make the file publicly accessible
    drive_service.permissions().create(
        fileId=file_id,
        body={'type': 'anyone', 'role': 'reader'}
    ).execute()
    return f"https://drive.google.com/uc?id={file_id}"

# ---------------------------
# BUILD IMAGE FORMULAS FOR GOOGLE SHEETS
# ---------------------------
# For each chart image file, upload it and create an IMAGE formula.
# The formula uses mode 4 (custom size) with height=800 and width=1600.
rows = []
for chart_path in chart_files:
    image_url = upload_image_to_drive(chart_path)
    # Build the IMAGE formula. Using value_input_option 'USER_ENTERED'
    formula = f'=IMAGE("{image_url}", 4, 960, 1600)'
    rows.append([formula])

# ---------------------------
# UPDATE GOOGLE SHEET WITH THE IMAGE FORMULAS
# ---------------------------
# Using value_input_option='USER_ENTERED' so the formula is interpreted properly (without a leading apostrophe)

sheet.clear()
sheet.update("A1", rows, value_input_option='USER_ENTERED')

print("Successfully updated the Google Sheet with chart images.")
